In [ ]:
import functools

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import nengo
import nengo.utils.numpy as npext
import nengo_gui.ipython

# Flexible motor oscillators

Goal: Make an oscillator for a syllable
with as few neurons as possible.
Must be able to oscillate at ~2-5 Hz.

In [ ]:
from collections import namedtuple

# Speech action indices
(lab_clos_full, lab_clos_fric,
 api_clos_full, api_clos_fric,
 api_clos_lat, dor_clos_full,
 glott_adduc_phon, glott_abduc_nophon,
 vow_ii, vow_aa, vow_uu) = list(range(11))

# Articulator indices
(tongue_dors_hl, tongue_dors_fb,
 lips_protrusion, lips_constriction,
 ttip_constriction, tdor_constriction) = list(range(6))

# --- Speech Actions to decode from the oscillator
SA = namedtuple('SA', ['onset', 'offset', 'idx'])

def sa_func(name, actions):
    def _fn(x):
        ret = np.zeros(11)
        for action in actions:
            if action.onset <= x <= action.offset:
                ret[action.idx] = 1.0
        return ret
    _fn.__name__ = name
    _fn.actions = actions  # Sneakily store actions
    return _fn

bas_sa = [SA(0.2, 0.4, lab_clos_full),
          SA(0.5, 0.7, api_clos_fric),
          SA(0.2, 0.6, glott_adduc_phon),
          SA(0.5, 0.7, glott_abduc_nophon),
          SA(0.2, 0.6, vow_aa)]
kul_sa = [SA(0.5, 0.7, api_clos_lat),
          SA(0.2, 0.4, dor_clos_full),
          SA(0.5, 0.7, glott_adduc_phon),
          SA(0.2, 0.5, glott_abduc_nophon),
          SA(0.2, 0.6, vow_uu)]
tip_sa = [SA(0.5, 0.7, lab_clos_full),
          SA(0.2, 0.4, api_clos_full),
          SA(0.2, 0.6, glott_adduc_phon),
          SA(0.5, 0.7, glott_abduc_nophon),
          SA(0.2, 0.6, vow_ii)]

bas = sa_func('bas', bas_sa)
kul = sa_func('kul', kul_sa)
tip = sa_func('tip', tip_sa)

In [ ]:
def plot_sa(sa_f):
    x = np.linspace(0, 1, 100)
    out = np.zeros((100, 11))
    for i, xx in enumerate(x):
        out[i] = sa_f(xx)
    plt.plot(x, out)
    plt.ylim(-0.1, 1.1)

plt.figure(figsize=(10, 5))
plt.subplot(3, 1, 1)
plot_sa(bas)
plt.subplot(3, 1, 2)
plot_sa(kul)
plt.subplot(3, 1, 3)
plot_sa(tip)

In [ ]:
from nengo.dists import Choice, Uniform

def zone(x):
    theta = np.arctan2(x[1], x[0])
    if theta > 0.75 * np.pi:
        return [0, 0]
    else:
        return x

def radial_f(fn):
    def _fn(x):
        # theta = np.arctan2(x[1], x[0])
        # t = theta / (2 * np.pi) + 0.5
        return fn(np.arctan2(x[1], x[0]) / (2 * np.pi) + 0.5)
    _fn.__name__ = fn.__name__
    return _fn

def Speech(n_neurons, net=None):
    if net is None:
        net = nengo.Network(label="Speech")

    with net:
        # Make a speech actions EA as a readout
        net.speechactions = nengo.networks.EnsembleArray(n_neurons, 11)
        # Oscillator kick -- this will be an ensemble eventually
        net.osc_kick = nengo.Node(output=lambda t: 0.7 if t % 0.18 < 0.05 else 0.0)
    net.output = net.speechactions.output
    return net

def Syllable(speech, syllable_f, tau=0.025, freq=3.3, net=None):
    if net is None:
        net = nengo.Network(label=syllable_f.__name__)

    n_neurons = speech.speechactions.n_neurons
    omega = tau * 2 * np.pi * freq
    encoders = [[np.cos(theta), np.sin(theta)]
                for theta in np.random.uniform(-np.pi, 0.75 * np.pi, n_neurons * 2)]

    with net:
        net.osc = nengo.Ensemble(n_neurons * 2, dimensions=2,
                                 intercepts=Uniform(0.3, 1),
                                 encoders=encoders,
                                 label=syllable_f.__name__)
        nengo.Connection(net.osc, net.osc,
                         transform=[[1, -omega], [omega, 1]], function=zone, synapse=tau)
        nengo.Connection(speech.osc_kick, net.osc, transform=[[-1], [0]])
        nengo.Connection(net.osc, speech.speechactions.input, function=radial_f(syllable_f))
        # By default, oscillator will be inhibited. Inhibit this to remove that.
        net.inhib = nengo.Ensemble(20, dimensions=1, intercepts=Uniform(-0.4, 0.1), encoders=Choice([[1]]))
        nengo.Connection(net.inhib.neurons, net.osc.neurons, transform=-1 * np.ones((n_neurons * 2, 20)))
    return net

with nengo.Network() as net:
    sp = Speech(n_neurons=200)
    s_bas = Syllable(sp, bas, freq=5.3)
    s_kul = Syllable(sp, kul, freq=2.5)
    s_tip = Syllable(sp, tip, freq=5.0)
    # disinhibit one to let it through
    disinhibit = nengo.Node(-1)
    nengo.Connection(disinhibit, s_bas.inhib)

    # Probes
    p_osc = nengo.Probe(s_bas.osc, synapse=0.01)
    p_sa = nengo.Probe(sp.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(0.4)

In [ ]:
t = sim.trange()
plt.figure()
plt.plot(sim.data[p_osc].T[0], sim.data[p_osc].T[1])
plt.figure()
plt.plot(t, sim.data[p_sa])
plt.xlim(right=t[-1])

In [ ]:
print(sum(ens.n_neurons for ens in net.all_ensembles))

## Speed control

Add in an intermediate population to control speed across all syllables.

In [ ]:
def zone(x):
    theta = np.arctan2(x[1], x[0])
    if np.pi < theta < 0:
        return 0
    else:
        return x

def radial_f(fn):
    def _fn(x):
        # theta = np.arctan2(x[1], x[0])
        # t = theta / (2 * np.pi) + 0.5
        return fn(np.arctan2(x[1], x[0]) / (2 * np.pi) + 0.5)
    _fn.__name__ = fn.__name__
    return _fn

def Speech(neurons_per_d, net=None):
    if net is None:
        net = nengo.Network(label="Speech")

    with net:
        # Make a speech actions EA as a readout
        net.speechactions = nengo.networks.EnsembleArray(neurons_per_d, 11)
        # Oscillator kick -- this will be an ensemble eventually
        net.osc_kick = nengo.Node(output=lambda t: 0.8 if t < 0.05 else 0.0)
        # Global speed control
        net.speed = nengo.Ensemble(neurons_per_d, dimensions=1)
    net.output = net.speechactions.output
    return net

def Syllable(speech, syllable_f, tau=0.01, net=None):
    if net is None:
        net = nengo.Network(label=syllable_f.__name__)

    neurons_per_d = speech.speechactions.n_neurons
    osc_neurons = neurons_per_d * 8
    encoders = [[np.cos(theta), np.sin(theta)]
                for theta in np.random.uniform(-np.pi, 0.5 * np.pi, osc_neurons)]

    def feedback(x, w_max=2*np.pi*4):
        x0, x1, w = x  # These are the three variables stored in the ensemble
        # w *= -1
        w += 1  # We offset w, so w=0 is normal speed (1.0)
        return zone(np.array([x0 - w*w_max*tau*x1, x1 + w*w_max*tau*x0]))
        
    with net:
        net.osc = nengo.Ensemble(osc_neurons, dimensions=2,
                                 intercepts=Uniform(0.3, 1),
                                 encoders=encoders,
                                 label=syllable_f.__name__)
        # Since osc uses special encoders and such, we want to do our control
        # in a separate ensemble. This adds a slight delay, but that's ok.
        ctrl = nengo.Ensemble(neurons_per_d * 3, dimensions=3, radius=1.7)
        nengo.Connection(net.osc, ctrl[:2], synapse=0.005)
        nengo.Connection(speech.speed, ctrl[2], synapse=0.005)
        nengo.Connection(ctrl, net.osc, function=feedback, synapse=tau)
        # Get kick input
        nengo.Connection(speech.osc_kick, net.osc, transform=[[-1], [0]])
        nengo.Connection(net.osc, speech.speechactions.input, function=radial_f(syllable_f))
        # By default, oscillator will be inhibited. Inhibit this to remove that.
        net.inhib = nengo.Ensemble(20, dimensions=1,
                                   intercepts=Uniform(-0.4, 0.1), encoders=Choice([[1]]))
        nengo.Connection(net.inhib.neurons, net.osc.neurons,
                         transform=-1 * np.ones((net.osc.n_neurons, 20)))
    return net


with nengo.Network() as net:
    speech = Speech(neurons_per_d=100)
    s_bas = Syllable(speech, bas)
    s_kul = Syllable(speech, kul)
    s_tip = Syllable(speech, tip)
    # disinhibit one to let it through
    disinhibit = nengo.Node(-1)
    nengo.Connection(disinhibit, s_bas.inhib)

    # Positive = speed up, negative = speed down, -1 = stop
    # Note: if you speed up too much, it'll repeat
    nengo.Connection(nengo.Node(0.1), speech.speed, synapse=None)

    # Probes
    p_osc = nengo.Probe(s_bas.osc, synapse=0.01)
    p_sa = nengo.Probe(speech.output, synapse=0.01)
    p_spd = nengo.Probe(speech.speed, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(0.4)

t = sim.trange()
plt.figure()
plt.plot(sim.data[p_osc].T[0], sim.data[p_osc].T[1])
plt.figure()
plt.plot(t, sim.data[p_spd])
plt.xlim(right=t[-1])
plt.figure()
plt.plot(t, sim.data[p_sa])
plt.xlim(right=t[-1])

In [ ]:
print(sum(ens.n_neurons for ens in net.all_ensembles))